In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as stats

from statsmodels.stats.multitest import fdrcorrection

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LinearSegmentedColormap

import dxpy
import os

In [ ]:
# Input and output files
DATA="/path/to/UKB/data.csv" # Use the output of script 3_Data preparation/UKB/6_compile_data.ipynb
OUTPUT_DIR="/path/to/output/directory"

# Output will be (1) a heatmap of the results, as shown in Fig. 6D (right) and 
# (2) a table of t-test results, as shown in Table S5E

In [ ]:
# Perform t-tests for the burden of specific variant classes with phenotypes
df=pd.read_csv(DATA)

In [ ]:
# Remove splice variants from All coding SNVs
df['All coding SNVs']=df['All coding SNVs']-df.Splice
df['All coding SNVs (LF)']=df['All coding SNVs (LF)']-df['Splice (LF)']

In [ ]:
vars=['All coding SNVs', 'Missense', 'LOF', 'Genes del.', 'Genes dup.',
      'Intelligence PRS', 'SCZ PRS', 'Education PRS', 'Autism PRS',
      'All coding SNVs (LF)', 'Missense (LF)', 'LOF (LF)', 'Genes del. (LF)', 'Genes dup. (LF)']
phenos=['Depression', 'Anxiety', 'Sleep trouble', 'Psychosis', 'Addiction', 'Mood lability']

In [ ]:
def cohens_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

In [ ]:
stat_lst=[]
for v in vars:
    for p in phenos:
        subdf=df[(~df[v].isnull()) & (~df[p].isnull())]

        # Restrict analyses to phenotype/variant combinations where at least 5 or 10% of the cohort are available as cases and controls
        cutoff=subdf.shape[0]*0.1
        if cutoff<5:
            cutoff=5

        x=subdf[subdf[p]>0][v].to_numpy()
        y=subdf[subdf[p]==0][v].to_numpy()

        if len(x)<cutoff or len(y)<cutoff:
            continue

        res=stats.ttest_ind(x, y, alternative='two-sided')

        # Save stats
        stat_lst.append([p, v, 'Two tailed t-test', np.mean(x), np.std(x), len(x), np.mean(y), np.std(y), len(y), cohens_d(x, y), res.statistic, res.pvalue])
statdf=pd.DataFrame(stat_lst, columns=['Phenotype', 'Variant', 'Test', 'Case mean', 'Case SD', 'Case N', 'Control mean', 'Control SD', 'Control N', "Cohen's D", 'statistic', 'p value'])

In [ ]:
statdf[statdf['p value']<=0.05]

In [ ]:
# Multiple test correct
# Correct separately over LF and non-LF categories
statdf['BH FDR']=np.nan
statdf.loc[(statdf.Variant.str.contains('LF')) & (~statdf['p value'].isnull()), 'BH FDR']=fdrcorrection(statdf[(statdf.Variant.str.contains('LF')) & (~statdf['p value'].isnull())]['p value'].to_numpy())[1]
statdf.loc[(~statdf.Variant.str.contains('LF')) & (~statdf['p value'].isnull()), 'BH FDR']=fdrcorrection(statdf[(~statdf.Variant.str.contains('LF')) & (~statdf['p value'].isnull())]['p value'].to_numpy())[1]

In [ ]:
# Save results
statdf.to_csv('UKB_ttests.csv', index=False)
dxpy.upload_local_file('UKB_ttests.csv', folder=OUTPUT_DIR, parents=True)
os.remove('UKB_ttests.csv')

In [ ]:
# Make a heatmap of results
plotdf=statdf.pivot(index='Variant', columns='Phenotype', values="Cohen's D")
statdf['star']=''
statdf.loc[statdf['p value']<=0.05, 'star']='*'
statdf.loc[statdf['BH FDR']<=0.05, 'star']='**'
stardf=statdf.pivot(index='Variant', columns='Phenotype', values="star")

pheno_order=['Depression', 'Anxiety', 'Sleep trouble', 'Mood lability']

plotdf=plotdf.loc[vars, pheno_order]
stardf=stardf.loc[vars, pheno_order]

biggest=1.4

colors=["#053061", "#2166AC", "#4393C3", "#92C5DE", "#D1E5F0", '#FFFFFF', "#FDDBC7", "#F4A582", "#D6604D", "#B2182B", "#67001F"]
cmap=LinearSegmentedColormap.from_list('BlRd', colors, N=20)
cmap.set_bad('#CCCCCC')
sns.heatmap(data=plotdf, cmap=cmap, vmin=-biggest, vmax=biggest, square=True, fmt='', linecolor='k', linewidths=0.75, annot=stardf)
plt.tight_layout()
plt.savefig('Fig6D_right.pdf')
plt.close()

In [ ]:
# Save plot
dxpy.upload_local_file('Fig6D_right.pdf', folder=OUTPUT_DIR, parents=True)
os.remove('Fig6D_right.pdf')